In [8]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import pandas as pd
import json

Load Configuration File

In [19]:
with open('config.json') as config_file:
    config = json.load(config_file)
    print(config)

{'database': {'name': 'BikeStores', 'server': '.\\SQLEXPRESS', 'driver': 'SQL Server'}, 'rl_contexts': [{'query': 'SELECT * FROM production.products WHERE [INPUT]', 'table_filter': ['production.products'], 'column_filter': ['product_name'], 'goal': 'Trek 820 - 2016'}]}


Set Up Connection to Database

In [17]:
db = config['database']

name = db['name']
server = db['server']
driver = db['driver']

# Connect to SQL database using the above parameters.
conn_string = f'DRIVER={driver};SERVER={server};DATABASE={name};Trusted_Connection=yes'
conn_url = URL.create('mssql+pyodbc', query={'odbc_connect': conn_string})
engine = create_engine(conn_url)

# Display a dataframe from a sample query.
sample_query = 'SELECT * FROM production.brands'
df = pd.read_sql(sample_query, engine)
df

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey
5,6,Strider
6,7,Sun Bicycles
7,8,Surly
8,9,Trek
